In [12]:
import requests
import pandas as pd

API_KEY = 'a072e52a322008c1adbb324e8ea12fbb3852a28d'
YEAR = '2022'
BASE_URL = f'https://api.census.gov/data/{YEAR}/acs/acs5'

#interested variables
variables = {
    'B19301_001E': 'Per_Capita_Income',
    'B19013_001E': 'Median_Household_Income',
    'B23025_005E': 'Unemployed',
    'B23025_003E': 'Labor_Force',
    'B17001_002E': 'Below_Poverty',
    'B17001_001E': 'Total_Poverty_Universe',
    'B25077_001E': 'Median_Home_Value',
    'B25064_001E': 'Median_Gross_Rent'
}

var_string = ",".join(variables.keys())

#pull code
url = f"{BASE_URL}?get=NAME,{var_string}&for=zip%20code%20tabulation%20area:*&key={API_KEY}"
response = requests.get(url)
#print(f"Status Code: {response.status_code}")

#cast into pandas data frame
data = response.json()
columns = data[0] #first row
rows = data[1:] #rest rows
df = pd.DataFrame(rows, columns=columns)

#rename columns and convert to float
df.rename(columns=variables, inplace=True)
for col in variables.values():
    df[col] = pd.to_numeric(df[col], errors='coerce')

#calculate  unemployment and poverty rate
df['Unemployment_Rate'] = (df['Unemployed'] / df['Labor_Force']) * 100
df['Poverty_Rate'] = (df['Below_Poverty'] / df['Total_Poverty_Universe']) * 100

#filter for CO(80-81), UT(84) with zip code prefixes
df = df[df['zip code tabulation area'].str[:2].isin(['80', '81', '84'])]



df.to_csv('CO_UT_Zipcode_Employment_Economics.csv', index=False)